In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import plotly.graph_objects as go
import os

In [ ]:
import sys
# sys.path.append('/content/drive/MyDrive/PFC/Vacancy')

import importlib

import PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
beta_val = 1.5
epsilon_val = -4.0 #-2.82
g_val = 1.
phi_s = 0.5 #0.54

In [ ]:
# simulation
steps = int(5e3)

sim_dxdy = PFC2D_Vacancy.PFC2D_Vacancy()
sim_dxdy.parms.epsilon = epsilon_val
sim_dxdy.parms.a = 0
sim_dxdy.parms.beta = beta_val
sim_dxdy.parms.g = -g_val
sim_dxdy.parms.v0 = 1
sim_dxdy.parms.phi0 = phi_s

sim_dxdy.parms.Hng = 0
sim_dxdy.parms.Hln = 1
sim_dxdy.parms.N = 360
sim_dxdy.parms.PPU = 30
sim_dxdy.parms.eta = 0.01
sim_dxdy.parms.dt = 0.01
sim_dxdy.parms.seed = 34125
sim_dxdy.parms.NoiseDynamicsFlag = False
sim_dxdy.parms.Noise_CutoffK = 0.8
sim_dxdy.parms.NoiseTimeSmoothFlag = True
sim_dxdy.parms.NoiseTimeSmoothingFrames = 0
sim_dxdy.parms.Noise_CutoffOmega = 0.04

qRange = np.linspace(0.900, 1.010, 7, endpoint=True)
groundState = np.zeros_like(qRange)
for qi, q in enumerate(qRange):
  sim_dxdy.InitParms()
  sim_dxdy.SetGeometry(100, 180, 100, scalefactor = 1/q, forceUnitCellBoundary=False)
  sim_dxdy.InitFieldCrystal(noisy=True, scalefactor = 1/q)
  sim_dxdy.t = 0

  # Total steps and checkpoints
  total_steps = steps if steps is not None else int(1e6)
  # checkpoints = 10
  energy_calcs = 10
  # visualizations = 1
  # particle_trackings = 10

  # fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi, colorbar_thickness=10))
  # fig.update_layout(
  #     yaxis=dict(scaleanchor="x"),
  #     width=sim_dxdy.nx*2 + 100,
  #     height=sim_dxdy.ny*2 + 100,
  #     margin=dict(l=0, r=0, t=0, b=0),
  #     xaxis_showgrid=False,
  #     yaxis_showgrid=False,
  #     xaxis_zeroline=False,
  #     yaxis_zeroline=False,
  #     xaxis_visible=False,
  #     yaxis_visible=False,
  #     plot_bgcolor='rgba(0,0,0,0)',
  #     title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
  # )
  # fig.show()

  # Run sim_dxdyulation
  for i in range(total_steps):

    try:

      # Save checkpoint
      # if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      #   cpname = os.path.join(outputPath, f'_cp_{i//int(total_steps/checkpoints):04}')
      #   sim_dxdy.Save(cpname)
      #   print(f'Saved {cpname}')

      # Show energy, progress
      if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
        sim_dxdy.CalcEnergyDensity()
        print(f'qi: {qi}, q: {q:.4f}, f: {sim_dxdy.f:.10f}, t: {sim_dxdy.t:.2f}, min: {sim_dxdy.phi.min()}, max: {sim_dxdy.phi.max()}, mean: {sim_dxdy.phi.mean()}')

      # Write visualization
      # if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      #   fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi, colorbar_thickness=10))
      #   fig.update_layout(
      #       yaxis=dict(scaleanchor="x"),
      #       width=sim_dxdy.nx*2 + 100,
      #       height=sim_dxdy.ny*2 + 100,
      #       margin=dict(l=0, r=0, t=0, b=0),
      #       xaxis_showgrid=False,
      #       yaxis_showgrid=False,
      #       xaxis_zeroline=False,
      #       yaxis_zeroline=False,
      #       xaxis_visible=False,
      #       yaxis_visible=False,
      #       plot_bgcolor='rgba(0,0,0,0)',
      #       title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      #   )
      #   fig.show()

      # Calculate CM
      # if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      #   max_phi = float(sim_dxdy.phi.max()/3)
      #   print (f'Calculating CM with threshold {max_phi}')
      #   cm_data = sim_dxdy.SPT_LocateCM(nxy=sim_dxdy.parms.PPU, min_distance=sim_dxdy.parms.PPU//2, threshold_abs=max_phi)
      #   print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      #   sim_dxdy.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
      #   print(f'Saved cm.h5 with {len(cm_data)} entries')

      # Step sim_dxdyulation
      sim_dxdy.TimeStepCross()
    except Exception as e:
      print(e)
      break

  fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi.get(), colorbar_thickness=10))
  fig.update_layout(
      yaxis=dict(scaleanchor="x"),
      width=sim_dxdy.nx*2 + 100,
      height=sim_dxdy.ny*2 + 100,
      margin=dict(l=0, r=0, t=0, b=0),
      xaxis_showgrid=False,
      yaxis_showgrid=False,
      xaxis_zeroline=False,
      yaxis_zeroline=False,
      xaxis_visible=False,
      yaxis_visible=False,
      plot_bgcolor='rgba(0,0,0,0)',
      title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
  )
  fig.show()
  groundState[qi] = sim_dxdy.f

In [ ]:
# plot ground state energy vs. q
fig = go.Figure(data=go.Scatter(x=qRange, y=groundState, mode='lines+markers'))
fig.update_layout(
    width=800,
    height=500,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_title="q",
    yaxis_title="f"
)
fig.show()

qCoeffs = np.polyfit(qRange, groundState, 4)
qPoly = np.poly1d(qCoeffs)
dqPoly = np.polyder(qPoly)
dqRoots = np.roots(dqPoly)
dqRoots = dqRoots[np.isreal(dqRoots)]
print(dqRoots)

In [ ]:
# simulation
steps = int(5e3)

sim_dxdy = PFC2D_Vacancy.PFC2D_Vacancy()
sim_dxdy.parms.epsilon = epsilon_val
sim_dxdy.parms.a = 0
sim_dxdy.parms.beta = beta_val
sim_dxdy.parms.g = -g_val
sim_dxdy.parms.v0 = 1
sim_dxdy.parms.phi0 = phi_s

sim_dxdy.parms.Hng = 0
sim_dxdy.parms.Hln = 1
sim_dxdy.parms.N = 360
sim_dxdy.parms.PPU = 30
sim_dxdy.parms.eta = 0.01
sim_dxdy.parms.dt = 0.005
sim_dxdy.parms.seed = 34125
sim_dxdy.parms.NoiseDynamicsFlag = False
sim_dxdy.parms.Noise_CutoffK = 0.8
sim_dxdy.parms.NoiseTimeSmoothFlag = True
sim_dxdy.parms.NoiseTimeSmoothingFrames = 0
sim_dxdy.parms.Noise_CutoffOmega = 0.04

qRange = np.linspace(0.850, 0.950, 7, endpoint=True)
groundState = np.zeros_like(qRange)
for qi, q in enumerate(qRange):
  sim_dxdy.InitParms()
  sim_dxdy.SetGeometry(160, 144, 20, scalefactor = 1/q, forceUnitCellBoundary=True)
  sim_dxdy.InitFieldCrystal(noisy=True, scalefactor = 1/q)
  sim_dxdy.t = 0

  # Total steps and checkpoints
  total_steps = steps if steps is not None else int(1e6)
  # checkpoints = 10
  energy_calcs = 10
  # visualizations = 1
  # particle_trackings = 10

  # fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi, colorbar_thickness=10))
  # fig.update_layout(
  #     yaxis=dict(scaleanchor="x"),
  #     width=sim_dxdy.nx*2 + 100,
  #     height=sim_dxdy.ny*2 + 100,
  #     margin=dict(l=0, r=0, t=0, b=0),
  #     xaxis_showgrid=False,
  #     yaxis_showgrid=False,
  #     xaxis_zeroline=False,
  #     yaxis_zeroline=False,
  #     xaxis_visible=False,
  #     yaxis_visible=False,
  #     plot_bgcolor='rgba(0,0,0,0)',
  #     title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
  # )
  # fig.show()

  # Run sim_dxdyulation
  for i in range(total_steps):

    try:

      # Save checkpoint
      # if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      #   cpname = os.path.join(outputPath, f'_cp_{i//int(total_steps/checkpoints):04}')
      #   sim_dxdy.Save(cpname)
      #   print(f'Saved {cpname}')

      # Show energy, progress
      if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
        sim_dxdy.CalcEnergyDensity()
        print(f'qi: {qi}, q: {q:.4f}, f: {sim_dxdy.f:.10f}, t: {sim_dxdy.t:.2f}, min: {sim_dxdy.phi.min()}, max: {sim_dxdy.phi.max()}, mean: {sim_dxdy.phi.mean()}')

      # Write visualization
      # if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      #   fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi, colorbar_thickness=10))
      #   fig.update_layout(
      #       yaxis=dict(scaleanchor="x"),
      #       width=sim_dxdy.nx*2 + 100,
      #       height=sim_dxdy.ny*2 + 100,
      #       margin=dict(l=0, r=0, t=0, b=0),
      #       xaxis_showgrid=False,
      #       yaxis_showgrid=False,
      #       xaxis_zeroline=False,
      #       yaxis_zeroline=False,
      #       xaxis_visible=False,
      #       yaxis_visible=False,
      #       plot_bgcolor='rgba(0,0,0,0)',
      #       title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      #   )
      #   fig.show()

      # Calculate CM
      # if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      #   max_phi = float(sim_dxdy.phi.max()/3)
      #   print (f'Calculating CM with threshold {max_phi}')
      #   cm_data = sim_dxdy.SPT_LocateCM(nxy=sim_dxdy.parms.PPU, min_distance=sim_dxdy.parms.PPU//2, threshold_abs=max_phi)
      #   print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      #   sim_dxdy.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
      #   print(f'Saved cm.h5 with {len(cm_data)} entries')

      # Step sim_dxdyulation
      sim_dxdy.TimeStepCross()
    except Exception as e:
      print(e)
      break

  fig = go.Figure(data=go.Heatmap(z=sim_dxdy.phi.get(), colorbar_thickness=10))
  fig.update_layout(
      yaxis=dict(scaleanchor="x"),
      width=sim_dxdy.nx*2 + 100,
      height=sim_dxdy.ny*2 + 100,
      margin=dict(l=0, r=0, t=0, b=0),
      xaxis_showgrid=False,
      yaxis_showgrid=False,
      xaxis_zeroline=False,
      yaxis_zeroline=False,
      xaxis_visible=False,
      yaxis_visible=False,
      plot_bgcolor='rgba(0,0,0,0)',
      title=dict(text=f"t: {sim_dxdy.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
  )
  fig.show()
  groundState[qi] = sim_dxdy.f

In [ ]:
# plot ground state energy vs. q
fig = go.Figure(data=go.Scatter(x=qRange, y=groundState, mode='lines+markers'))
fig.update_layout(
    width=800,
    height=500,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_title="q",
    yaxis_title="f"
)
fig.show()

qCoeffs = np.polyfit(qRange, groundState, 4)
qPoly = np.poly1d(qCoeffs)
dqPoly = np.polyder(qPoly)
dqRoots = np.roots(dqPoly)
dqRoots = dqRoots[np.isreal(dqRoots)]
print(dqRoots)

In [ ]:
# simulation
steps = int(6e3)

q_val = 0.918   ## energy min: 0.98168487  ## relaxation of 640x576/20: 0.95427
phi_s = 0.50
phi_l = 0.15

sim_v = PFC2D_Vacancy.PFC2D_Vacancy()
sim_v.parms.epsilon = epsilon_val
sim_v.parms.a = 0
sim_v.parms.beta = beta_val
sim_v.parms.g = -g_val
sim_v.parms.v0 = 1
sim_v.parms.phi0 = phi_s

sim_v.parms.Hng = 0
sim_v.parms.Hln = 1
sim_v.parms.N = 360
sim_v.parms.PPU = 30
sim_v.parms.eta = 0.01
sim_v.parms.dt = 0.01
sim_v.parms.seed = 34125
sim_v.parms.NoiseDynamicsFlag = False
sim_v.parms.Noise_CutoffK = 0.8
sim_v.parms.NoiseTimeSmoothFlag = True
sim_v.parms.NoiseTimeSmoothingFrames = 0
sim_v.parms.Noise_CutoffOmega = 0.04

sim_v.InitParms()
sim_v.SetGeometry(640, 576, 20, scalefactor = 1, forceUnitCellBoundary = False)
sim_v.InitFieldCrystal(A = sim_v.parms.phi0/3, noisy=True, scalefactor = 1/q_val)
sim_v.phi[:,:int((2.85)*20/q_val)] = phi_l
sim_v.phi[:,int((sim_v.mx - 4.5)*20/q_val):] = phi_l
sim_v.phi[:int((4 + 2/3)*20*np.sqrt(3)/2/q_val),:] = phi_l
sim_v.phi[int((int(sim_v.my/2)*2 - 4 + 1/3)*20*np.sqrt(3)/2/q_val):,:] = phi_l
sim_v.t = 0

# Total steps and checkpoints
total_steps = steps if steps is not None else int(1e6)
checkpoints = 1
energy_calcs = 10
visualizations = 1
particle_trackings = 1
outputPath = './island'
# create output directory if it doesn't exist
if not os.path.exists(outputPath):
  os.makedirs(outputPath)

fig = go.Figure(data=go.Heatmap(z=sim_v.phi.get(), colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=sim_v.nx*2 + 100,
    height=sim_v.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {sim_v.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()

# Run sim_vulation
for i in range(total_steps):

  try:

    # Save checkpoint
    if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      cpname = os.path.join(outputPath, f'_cp_{i//int(total_steps/checkpoints):04}')
      sim_v.Save(cpname)
      print(f'Saved {cpname}')

    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_v.CalcEnergyDensity()
      print(f'qi: {qi}, q: {q:.4f}, f: {sim_v.f:.10f}, t: {sim_v.t:.2f}, min: {sim_v.phi.min()}, max: {sim_v.phi.max()}, mean: {sim_v.phi.mean()}')

    # Write visualization
    if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      fig = go.Figure(data=go.Heatmap(z=sim_v.phi.get(), colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=sim_v.nx*2 + 100,
          height=sim_v.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {sim_v.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.show()

    # Calculate CM
    if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      max_phi = float(sim_v.phi.max()/3)
      print (f'Calculating CM with threshold {max_phi}')
      cm_data = sim_v.SPT_LocateCM(nxy=sim_v.parms.PPU, min_distance=sim_v.parms.PPU//2, threshold_abs=max_phi)
      print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      sim_v.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
      print(f'Saved cm.h5 with {len(cm_data)} entries')

    # Step sim_vulation
    sim_v.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
# Total steps and checkpoints
total_steps = int(2e5)
checkpoints = 20
energy_calcs = 20
visualizations = 20
particle_trackings = 200

# Run sim_vulation
for i in range(total_steps):

  try:

    # Save checkpoint
    if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      cpname = os.path.join(outputPath, f'_cp_relax_{i//int(total_steps/checkpoints):04}')
      sim_v.Save(cpname)
      print(f'Saved {cpname}')

    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_v.CalcEnergyDensity()
      print(f'qi: {qi}, q: {q:.4f}, f: {sim_v.f:.10f}, t: {sim_v.t:.2f}, min: {sim_v.phi.min()}, max: {sim_v.phi.max()}, mean: {sim_v.phi.mean()}')

    # Write visualization
    if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      fig = go.Figure(data=go.Heatmap(z=sim_v.phi.get(), colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=sim_v.nx*2 + 100,
          height=sim_v.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {sim_v.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.show()

    # Calculate CM
    if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      max_phi = float(sim_v.phi.max()/3)
      print (f'Calculating CM with threshold {max_phi}')
      cm_data = sim_v.SPT_LocateCM(nxy=sim_v.parms.PPU, min_distance=sim_v.parms.PPU//2, threshold_abs=max_phi)
      print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      sim_v.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
      print(f'Saved cm.h5 with {len(cm_data)} entries')

    # Step sim_vulation
    sim_v.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
phi_island = sim_v.phi.get()

In [ ]:
nMax = 15
import copy
import cupy as cp
n = nMax
sim_v_n_list = [copy.deepcopy(sim_v) for _ in range(nMax)]
cx, cy = 314, 290
mxr = int((n - 1)/2) + ((n + 1)%2)
mxl = int((n - 1)/2)
sim_v_n = sim_v_n_list[n-1]
sim_v_n.phi = cp.array(phi_island)


In [ ]:
sim_v_n.phi[290-10:290+10, 314 - int((20 + 20*mxl)/q):314 + int((20 + 20*mxr)/q)] = 0

max_phi = float(sim_v_n.phi.max()/3)
print (f'Calculating CM with threshold {max_phi}')
cm_data = sim_v_n.SPT_LocateCM(nxy=sim_v_n.parms.PPU, min_distance=sim_v_n.parms.PPU//2, threshold_abs=max_phi)
print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,1].min()},{cm_data[:,0].min()}, max: {cm_data[:,1].max()},{cm_data[:,0].max()}')
nAtoms = len(cm_data)

fig = go.Figure(data=go.Heatmap(z=sim_v_n.phi.get(), colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=sim_v.nx*2 + 100,
    height=sim_v.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {sim_v.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()

In [ ]:
# Total steps and checkpoints
total_steps = int(1e5)
checkpoints = 1
energy_calcs = 10
visualizations = 5
particle_trackings = 10
outputPath = f'./island_n{n}'
# create output directory if it doesn't exist
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

# Run sim_v_nulation
for i in range(total_steps):

  try:
    # Save checkpoint
    if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      cpname = os.path.join(outputPath, f'_cp_{i//int(total_steps/checkpoints):04}')
      sim_v_n.Save(cpname)
      print(f'Saved {cpname}')

    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_v_n.CalcEnergyDensity()
      print(f'qi: {qi}, q: {q:.4f}, f: {sim_v_n.f:.10f}, t: {sim_v_n.t:.2f}, min: {sim_v_n.phi.min()}, max: {sim_v_n.phi.max()}, mean: {sim_v_n.phi.mean()}')

    # Write visualization
    if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      fig = go.Figure(data=go.Heatmap(z=sim_v_n.phi.get(), colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=sim_v_n.nx*2 + 100,
          height=sim_v_n.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {sim_v_n.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.show()

    # Calculate CM
    if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      max_phi = float(sim_v_n.phi.max()/3)
      print (f'Calculating CM with threshold {max_phi}')
      cm_data = sim_v_n.SPT_LocateCM(nxy=sim_v_n.parms.PPU, min_distance=sim_v_n.parms.PPU//2, threshold_abs=max_phi)
      print(f'CM data: {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      sim_v_n.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
      print(f'Saved cm.h5 with {len(cm_data)} entries')
      if len(cm_data) != nAtoms:
        sim_v_n.phi[290-10:290+10, 314 - int((20 + 20*mxl)/q):314 + int((20 + 20*mxr)/q)] = 0
        print('Vacancy region re-zeroed')

    # Step sim_v_nulation
    sim_v_n.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
sim_v_t = copy.deepcopy(sim_v_n)

In [ ]:
# Thermalization
eta_range = np.logspace(-3, -1, 20, endpoint=True)
sim_v_t.parms.NoiseDynamicsFlag = True
sim_v_t.parms.Noise_CutoffK = 0.8
sim_v_t.parms.NoiseTimeSmoothingFrames = 5
sim_v_t.parms.NoiseChangeRate = 3

# Total steps and checkpoints
total_steps = int(1e5)
checkpoints = 20
energy_calcs = 20
visualizations = 20
movie_frames = 2000
particle_trackings = 2000
thermalizations = len(eta_range)
outputPath = f'./island_n{n}_Thermalize'
# create output directory if it doesn't exist
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

# Run sim_v_tulation
for i in range(total_steps):

  try:
    if i % int(total_steps/thermalizations) == 0:
      sim_v_t.parms.eta = eta_range[i // int(total_steps/thermalizations)]
      sim_v_t.InitializeTimeSmoothNoise()
      print(f'Changed eta to {sim_v_t.parms.eta}')
      if not os.path.exists(os.path.join(outputPath, f'eta{sim_v_t.parms.eta:.4f}')):
        os.makedirs(os.path.join(outputPath, f'eta{sim_v_t.parms.eta:.4f}'))

    # Save checkpoint
    if i % int(total_steps/checkpoints) == int(total_steps/checkpoints)-1:
      cpname = os.path.join(outputPath, f'_cp_{i//int(total_steps/checkpoints):04}')
      sim_v_t.Save(cpname)
      print(f'Saved {cpname}')

    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim_v_t.CalcEnergyDensity()
      print(f'qi: {qi}, q: {q:.4f}, f: {sim_v_t.f:.10f}, t: {sim_v_t.t:.2f}, min: {sim_v_t.phi.min()}, max: {sim_v_t.phi.max()}, mean: {sim_v_t.phi.mean()}')

    # show visualization
    if i % int(total_steps/visualizations) == int(total_steps/visualizations)-1:
      fig = go.Figure(data=go.Heatmap(z=sim_v_t.phi.get(), colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=sim_v_t.nx*2 + 100,
          height=sim_v_t.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {sim_v_t.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.show()

    # write movie frames
    if i % int(total_steps/movie_frames) == int(total_steps/movie_frames)-1:
      fig = go.Figure(data=go.Heatmap(z=sim_v_t.phi.get(), colorbar_thickness=10))
      fig.update_layout(
          yaxis=dict(scaleanchor="x"),
          width=sim_v_t.nx*2 + 100,
          height=sim_v_t.ny*2 + 100,
          margin=dict(l=0, r=0, t=0, b=0),
          xaxis_showgrid=False,
          yaxis_showgrid=False,
          xaxis_zeroline=False,
          yaxis_zeroline=False,
          xaxis_visible=False,
          yaxis_visible=False,
          plot_bgcolor='rgba(0,0,0,0)',
          title=dict(text=f"t: {sim_v_t.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
      )
      fig.write_image(os.path.join(outputPath, f'eta{sim_v_t.parms.eta:.4f}', f'diffusion{i//int(total_steps/movie_frames):04}.png'))

    # Calculate CM
    if i % int(total_steps/particle_trackings) == int(total_steps/particle_trackings)-1:
      max_phi = float(sim_v_t.phi.max()/2)
      cm_data = sim_v_t.SPT_LocateCM(nxy=sim_v_t.parms.PPU, min_distance=sim_v_t.parms.PPU//2, threshold_abs=max_phi)
      print(f'CM data ({max_phi}): {len(cm_data)} entries, min: {cm_data[:,0].min()}, max: {cm_data[:,0].max()}')
      sim_v_t.SPT_AppendData(cm_data, os.path.join(outputPath, 'cm.h5'))
    #   print(f'Saved cm.h5 with {len(cm_data)} entries')
      if len(cm_data) != nAtoms:
        print('Number of atoms changed, stopping simulation')
        break

    # Step sim_v_tulation
    sim_v_t.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
sim_v_t.parms.eta

In [ ]:
fig = go.Figure(data=go.Heatmap(z=sim_v_t.noise.get(), colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=sim_v_n.nx*2 + 100,
    height=sim_v_n.ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)',
    title=dict(text=f"t: {sim_v_n.t:.2f}", font=dict(size=10), x=0.05, y=0.95, yref='paper')
)
fig.show()